Basic Algorithm to download images from google static api

In [15]:
import requests
import shutil
import os

def download_images(coordinates, zoom=18):
    url = 'https://maps.googleapis.com/maps/api/staticmap?'
    api_key = ''
    size = '500x500'
    scale = "1"
    maptype = 'satellite'

    response = requests.get(url + 'center=' + str(coordinates[0]) + "," + str(coordinates[1]) + '&zoom=' + str(zoom) + '&size=' + size + '&maptype=' + maptype + '&scale=' + scale + "&sensor=false"+ '&key=' + api_key, stream=True)
    with open(os.getcwd() + os.sep + 'Data' + os.sep + str(coordinates[0]) + "_" + str(coordinates[1]) + ".png", 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

Try to randomly pick coordinates and download the image

In [9]:
import random

def load_random_sample(n = 5):
  for i in range(n):
    random_lat = round(random.uniform(-90, 90), 3)
    random_lon = round(random.uniform(-180, 180), 3)
    download_images(coordinates=(random_lat,random_lon))

The problem of completly random image gathering is, that most places dont have exact satellite data and also the planet is 70% water, so you have a high likelihood of getting only images of oceans/seas.
The solution would be to give the randomiser tighter randomization bounderies.

Solution: Drawing "boxes" around countries and picking coordinates in those boxes.